# Load data

In [3]:
import joblib

base_models_and_predictions = joblib.load('models_and_predictions/v3_3732_3697_10fold.p')

In [5]:
base_models_and_predictions.keys()

dict_keys(['lgbm_1', 'cat_1', 'lgbm_2', 'cat_2', 'lgbm_1_oof_predictions', 'cat_1_oof_predictions', 'lgbm_2_oof_predictions', 'cat_2_oof_predictions'])

In [4]:
import polars as pl

organizer_df = pl.read_csv('../data/from_organizers/train.csv').to_pandas()
targets = organizer_df['utility_agent1'].values

# Compute baseline model scores

In [8]:
from sklearn.metrics import root_mean_squared_error

lgbm_1_predictions = base_models_and_predictions['lgbm_1_oof_predictions']
lgbm_1_rmse = root_mean_squared_error(targets, lgbm_1_predictions)
print(f'LightGBM 1 RMSE: {lgbm_1_rmse}')

lgbm_2_predictions = base_models_and_predictions['lgbm_2_oof_predictions']
lgbm_2_rmse = root_mean_squared_error(targets, lgbm_2_predictions)
print(f'LightGBM 2 RMSE: {lgbm_2_rmse}')

cat_1_predictions = base_models_and_predictions['cat_1_oof_predictions']
cat_1_rmse = root_mean_squared_error(targets, cat_1_predictions)
print(f'\nCatBoost 1 RMSE: {cat_1_rmse}')

cat_2_predictions = base_models_and_predictions['cat_2_oof_predictions']
cat_2_rmse = root_mean_squared_error(targets, cat_2_predictions)
print(f'CatBoost 2 RMSE: {cat_2_rmse}')

LightGBM 1 RMSE: 0.3732372449963131
LightGBM 2 RMSE: 0.3742012813900623

CatBoost 1 RMSE: 0.37490711099721374
CatBoost 2 RMSE: 0.37056349460727467


# Compute baseline + Isotonic scores

In [20]:
import numpy as np
from sklearn.model_selection import GroupKFold
from cir_model import CenteredIsotonicRegression

MODEL_NAMES = [
    'lgbm_1',
    # 'lgbm_2',
    'cat_1',
    # 'cat_2'
]
for model_name in MODEL_NAMES:
    oof_predictions = base_models_and_predictions[f'{model_name}_oof_predictions']
    oof_predictions = np.clip(oof_predictions, -1, 1)
    
    rmses = []

    groups = organizer_df['GameRulesetName'].values
    kf = GroupKFold(n_splits=10)
    for train_index, test_index in kf.split(oof_predictions, targets, groups):
        train_predictions = oof_predictions[train_index]
        train_targets = targets[train_index]
        test_predictions = oof_predictions[test_index]
        test_targets = targets[test_index]
        
        cir = CenteredIsotonicRegression()
        cir.fit(train_predictions, train_targets)
        test_predictions = cir.predict(test_predictions)
        
        rmse = root_mean_squared_error(test_targets, test_predictions)
        rmses.append(rmse)

    print(f'{model_name} RMSE: {np.mean(rmses)}')

lgbm_1 RMSE: 0.37171697476059673
cat_1 RMSE: 0.37151282412972014


# Compute ensemble scores

In [21]:
rmses = []

groups = organizer_df['GameRulesetName'].values
kf = GroupKFold(n_splits=10)
for train_index, test_index in kf.split(oof_predictions, targets, groups):
    rescaled_predictions = []
    for model_name in MODEL_NAMES:
        oof_predictions = base_models_and_predictions[f'{model_name}_oof_predictions'].clip(-1, 1)
        
        train_predictions = oof_predictions[train_index]
        train_targets = targets[train_index]
        test_predictions = oof_predictions[test_index]
        test_targets = targets[test_index]

        cir = CenteredIsotonicRegression()
        cir.fit(train_predictions, train_targets)

        rescaled_predictions.append(cir.predict(test_predictions))

    rescaled_predictions = np.mean(rescaled_predictions, axis=0)
    rmse = root_mean_squared_error(test_targets, rescaled_predictions)
    rmses.append(rmse)

print(f'Ensemble RMSE: {np.mean(rmses)}')

Ensemble RMSE: 0.3681353597009843
